In this notebook, a clustering of the obtained texts will be done in order to verify the labels they were given originally at the moment of downloading.

# Imports

In [1]:
from sklearn.ensemble import RandomForestClassifier
from src.preparation.obtained_text_da import ObtainedTextDA

/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.8/site-packages/sqlalchemy/orm/util.py:104: SAWarning: The 'delete-orphan' cascade option requires 'delete'.
  util.warn(


# Data obtaining

In [5]:
ot_da = ObtainedTextDA()
texts = ot_da.select_all_as_dataframe()
texts.head()

,CNCADC,CNCAdd,CNCAll,CNCCaus,CNCLogic,CNCTemp,CRFANP1,CRFANPa,CRFAO1,CRFAOa,...,WRDPRP2p,WRDPRP2s,WRDPRP3p,WRDPRP3s,WRDVERB,category,cluster_grade,filename,grade,id
0,1.927354,1.037806,61.082283,4.892513,51.297257,1.927354,0.086735,0.030246,0.244898,0.074469,...,0.000000,2.223870,49.073388,58.561898,113.120830,Comunicación,None,Ruta_castellano_inicial.txt,1.0,1.0
1,1.282599,1.282599,64.557503,8.978196,51.303976,1.710133,0.013100,0.005898,0.091703,0.065860,...,0.000000,0.000000,29.072253,69.687901,96.194955,Comunicación,None,abecedario.txt,1.0,2.0
2,1.993024,0.996512,66.517190,6.228201,55.057299,2.242152,0.026515,0.016618,0.231061,0.100263,...,0.000000,0.000000,28.151470,52.815147,78.475336,Comunicación,None,aguita-vida.txt,1.0,3.0
3,2.695418,2.695418,72.776280,2.695418,37.735849,26.954178,0.000000,0.003916,0.019231,0.029548,...,0.000000,0.000000,75.471698,13.477089,13.477089,Comunicación,None,antes-muy-antes.txt,1.0,4.0
4,2.023547,1.655629,59.234731,5.886681,48.749080,0.919794,0.038202,0.009743,0.175281,0.046130,...,0.367918,5.334805,21.155261,59.418690,79.286240,Comunicación,None,antologia-de-poesia-para-ninos-y-ninas.txt,1.0,5.0


In [6]:
# Delete unnecessary columns
X = texts.drop(columns=['filename', 'category', 'cluster_grade', 'grade'])
X.dropna(inplace=True) # Null records will be deleted, as seen in the notebook of exploratory data analysis
X

,CNCADC,CNCAdd,CNCAll,CNCCaus,CNCLogic,CNCTemp,CRFANP1,CRFANPa,CRFAO1,CRFAOa,...,WRDNOUN,WRDPRO,WRDPRP1p,WRDPRP1s,WRDPRP2p,WRDPRP2s,WRDPRP3p,WRDPRP3s,WRDVERB,id
0,1.927354,1.037806,61.082283,4.892513,51.297257,1.927354,0.086735,0.030246,0.244898,0.074469,...,233.951075,52.631579,31.727205,3.706449,0.000000,2.223870,49.073388,58.561898,113.120830,1.0
1,1.282599,1.282599,64.557503,8.978196,51.303976,1.710133,0.013100,0.005898,0.091703,0.065860,...,232.150492,40.615648,7.268063,0.855066,0.000000,0.000000,29.072253,69.687901,96.194955,2.0
2,1.993024,0.996512,66.517190,6.228201,55.057299,2.242152,0.026515,0.016618,0.231061,0.100263,...,231.190832,34.130543,0.498256,1.743896,0.000000,0.000000,28.151470,52.815147,78.475336,3.0
3,2.695418,2.695418,72.776280,2.695418,37.735849,26.954178,0.000000,0.003916,0.019231,0.029548,...,212.938005,37.735849,2.695418,2.695418,0.000000,0.000000,75.471698,13.477089,13.477089,4.0
4,2.023547,1.655629,59.234731,5.886681,48.749080,0.919794,0.038202,0.009743,0.175281,0.046130,...,224.061810,38.447388,1.287712,8.462104,0.367918,5.334805,21.155261,59.418690,79.286240,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1.391450,0.939228,58.406095,5.878874,49.639962,0.556580,0.024795,0.005598,0.135532,0.013851,...,238.077017,44.839462,7.061606,13.879709,0.869656,7.792117,22.889345,73.085887,105.680593,292.0
288,3.102290,1.559715,56.406828,9.358289,40.998217,1.388318,0.038963,0.015879,0.190970,0.025472,...,238.979158,39.284245,7.335801,4.627725,0.737008,4.593446,28.246264,66.947758,89.349376,293.0
289,3.484193,1.878911,57.954359,8.591912,42.886590,1.112753,0.040149,0.016301,0.180934,0.025914,...,233.167332,41.956256,9.522246,5.855634,0.310111,4.067933,26.268265,70.614203,96.608840,294.0
290,2.263993,1.160534,57.570108,9.170123,44.138351,0.837107,0.036862,0.012648,0.160552,0.022459,...,230.014079,44.975458,11.262890,7.457859,0.646855,5.327042,21.289144,70.640387,97.256573,295.0
